In [14]:
import os
from pathlib import Path
import spectral
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage.morphology import remove_small_objects, binary_closing

In [15]:
# Example input files
bil_file = "C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Cropped Berry/Good Blueberry 169-210-FromSelection-ConvertFormat.bil"  # Replace with your .bil file path
output_dir = "C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Blue(Good 169-210)_crop"

# Load the hyperspectral image using spectral library
hsi_image = spectral.open_image(f"{bil_file}.hdr").load()

# Dimensions of the grid (6x7 for 42 blueberries)
rows, cols = 7, 6

# Get the image dimensions
image_height, image_width, bands = hsi_image.shape

# Calculate crop dimensions
crop_height = image_height // rows
crop_width = image_width // cols

# Create output directory if it doesn't exist
Path(output_dir).mkdir(exist_ok=True)

# Crop and save each blueberry
for row in range(rows):
    for col in range(cols):
        # Calculate the crop bounds
        top = row * crop_height
        bottom = (row + 1) * crop_height
        left = col * crop_width
        right = (col + 1) * crop_width
        
        # Extract the crop
        blueberry = hsi_image[top:bottom, left:right, :]
        
    # Save the cropped image
        filename = os.path.join(output_dir, f"blueberry_{row}_{col}.npy")
        np.save(filename, blueberry)

        # Optional: Print the dimensions of each crop for verification
        print(f"Saved: {filename} | Crop dimensions: {blueberry.shape}")

print(f"All blueberries cropped and saved in '{output_dir}'")

Saved: C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Blue(Good 169-210)_crop\blueberry_0_0.npy | Crop dimensions: (216, 225, 462)
Saved: C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Blue(Good 169-210)_crop\blueberry_0_1.npy | Crop dimensions: (216, 225, 462)
Saved: C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Blue(Good 169-210)_crop\blueberry_0_2.npy | Crop dimensions: (216, 225, 462)
Saved: C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Blue(Good 169-210)_crop\blueberry_0_3.npy | Crop dimensions: (216, 225, 462)
Saved: C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Blue(Good 169-210)_crop\blueberry_0_4.npy | Crop dimensions: (216, 225, 462)
Saved: C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Blue(Good 169-210)_crop\blueberry_0_5.npy | Crop dimensions: (216, 225, 462)
Saved: C:/Users/zarin/OneDrive/Des

In [16]:
# Input and output directories
input_dir = "C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Blue(Good 169-210)_crop"
masked_output_dir = "C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Masked_blue(Good 169-210)"

# Create output directory if it doesn't exist
Path(masked_output_dir).mkdir(exist_ok=True)

# Loop through all .npy files in the input directory
for file in os.listdir(input_dir):
    if file.endswith(".npy"):
        # Load the cropped HSI data
        crop_file_path = os.path.join(input_dir, file)
        hsi_data = np.load(crop_file_path)


        # Step 1: Focus on Specific Bands (Preprocessing or Standalone)
        # Select specific bands that highlight blueberries (e.g., bands 20 to 40)
        selected_band = hsi_data[:, :, 400]  # Use index 399 for the 400th band (zero-based indexing)

        # Step 2: Create a Binary Mask Using Thresholding
        # Example: Use Otsu's threshold or a median-based split
        threshold = np.percentile(selected_band, 70)  # Median value
        binary_mask = selected_band > threshold  # Keep pixels above the threshold

        # Step 3: Refine the Mask with Morphological Operations
        binary_mask = remove_small_objects(binary_mask, min_size = 10000)  # Remove small artifacts
        binary_mask = binary_closing(binary_mask)  # Fill small gaps

        # Step 4: Apply the Mask to the Original HSI Data
        masked_hsi = hsi_data * binary_mask[:, :, np.newaxis]
        
         # Save the masked hyperspectral data as a .npy file
        masked_npy_file = os.path.join(masked_output_dir, f"masked_{file}")
        np.save(masked_npy_file, masked_hsi)

        # Save a visual representation as a .png file
        # Use the sum of bands for a grayscale image
        visual_image = masked_hsi.sum(axis=-1)
        masked_png_file = os.path.join(masked_output_dir, f"masked_{Path(file).stem}.png")
        
        plt.figure(figsize=(6, 6))
        plt.imshow(visual_image, cmap="gray")
        plt.axis("off")
        plt.title(f"Masked Image: {Path(file).stem}")
        plt.savefig(masked_png_file, bbox_inches="tight", pad_inches=0)
        plt.close()

        # Log progress
        print(f"Processed and saved: {masked_npy_file} and {masked_png_file}")

print(f"All masked files saved in '{masked_output_dir}'.")





Processed and saved: C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Masked_blue(Good 169-210)\masked_blueberry_0_0.npy and C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Masked_blue(Good 169-210)\masked_blueberry_0_0.png
Processed and saved: C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Masked_blue(Good 169-210)\masked_blueberry_0_1.npy and C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Masked_blue(Good 169-210)\masked_blueberry_0_1.png
Processed and saved: C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Masked_blue(Good 169-210)\masked_blueberry_0_2.npy and C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Masked_blue(Good 169-210)\masked_blueberry_0_2.png
Processed and saved: C:/Users/zarin/OneDrive/Desktop/NSU/12th Semester/CSE 499/Datasets/Blueberry/Masked_blue(Good 169-210)\masked_blueberry_0_3.npy and C:/U